# Distribution by counties

This file contains calculations for all identified areas. The vectorization and filtering of areas smaller than 1 hectare were performed using QGIS to ensure accuracy and relevance.

In [1]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import os
import numpy as np
import fiona
from tqdm import tqdm
from shapely.geometry import shape, Polygon
from rasterio import features

# Counties preparation
[[Click here to turn back to the table of contents]](#Table-of-contents)

Loading base layer to see total number of pixels

In [2]:
#taking parameters from Estonian layer
path_scenario_1_r = '../../Thesis_2024_updates/Raster_Data/Scenario_1/'
file_name_whole_estonia_r = 'whole_estonia/Estonia_rasterized_0_1.tif'
raster_base_layer = os.path.join(path_scenario_1_r, file_name_whole_estonia_r)

with rasterio.open(raster_base_layer) as src:
    base_trans = src.transform
    base_profile = src.profile
    base_shape = src.shape
    base_data = src.read(1)
    
# Create a boolean mask where True represents pixels with value 1
mask = base_data == 1

# Count the number of True values in the mask
num_total_pixels = np.sum(mask)
total_area_ha = round((num_total_pixels * 16)/10000)

print("Number of pixels equal to 1:", num_total_pixels)
print("Total area of Estonia: {} ha".format(total_area_ha))

Number of pixels equal to 1: 2716698675
Total area of Estonia: 4346718 ha


Loading counties

In [3]:
#taking parameters from Estonian layer
#vector path (input)
file_name_counties = '../../Thesis_2024_updates/Vector_Data/counties/maakond_shp/maakond_20231101.shp'

Reading counties

In [4]:
# Load the shapefile with the counties
counties = gpd.read_file(file_name_counties)
counties.shape

(15, 3)

Preprocess counties

In [5]:
counties['name'] = counties['MNIMI'].apply(lambda x: x[:-8] + ' county')
counties.head()

,MNIMI,MKOOD,geometry,name
0,Saare maakond,0074,"MULTIPOLYGON (((455191.283 6404986.597, 455187...",Saare county
1,Viljandi maakond,0084,"MULTIPOLYGON (((621049.180 6453125.120, 621052...",Viljandi county
2,Hiiu maakond,0039,"MULTIPOLYGON (((418048.719 6506294.706, 418045...",Hiiu county
3,Harju maakond,0037,"MULTIPOLYGON (((504775.973 6570562.880, 504787...",Harju county
4,Lääne maakond,0056,"MULTIPOLYGON (((460628.287 6512871.566, 460630...",Lääne county


Calculating the areas of the counties

In [6]:
# Open the raster layer
with rasterio.open(raster_base_layer) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_county'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['county_area_ha'] = counties['num_pixels_county'].apply(lambda x: round((x * 16)/10000))

# Calculations
[[Click here to turn back to the table of contents]](#Table-of-contents)

In [7]:
file_name_suitable_lands = '../../Thesis_2024_updates/Subtraction_Results/Final_file.tif'

In [9]:
# Open the raster layer
with rasterio.open(file_name_suitable_lands) as src:
    affine = src.transform
    array = src.read(1)  # Read the first band
    
    # Define the statistic to calculate: count of pixels with value 1
    stats = zonal_stats(counties, array, affine=affine, stats=['count'], 
                        categorical=True, nodata=src.nodata)

    # Adding the results to the counties GeoDataFrame
    counties['num_pixels_'] = [s.get(1, 0) for s in stats]  # Get count of 1's, default to 0 if no 1's
    counties['suitable_area_ha'] = counties['num_pixels'].apply(lambda x: round((x * 16)/10000))

In [10]:
counties['percent_of_county'] = round((counties['suitable_area_ha']/counties['county_area_ha']) * 100, 1)
counties['percent_of_county_str'] = counties['percent_of_county'].apply(lambda x: str(x) + '%')

In [11]:
counties.head()

,MNIMI,MKOOD,geometry,name,num_pixels_county,county_area_ha,num_pixels_S1,S1_suitable_area_ha,percent_of_county,percent_of_county_str
0,Saare maakond,0074,"MULTIPOLYGON (((455191.283 6404986.597, 455187...",Saare county,183644942,293832,5074856,8120,2.8,2.8%
1,Viljandi maakond,0084,"MULTIPOLYGON (((621049.180 6453125.120, 621052...",Viljandi county,213756560,342010,4753528,7606,2.2,2.2%
2,Hiiu maakond,0039,"MULTIPOLYGON (((418048.719 6506294.706, 418045...",Hiiu county,64535957,103258,1687557,2700,2.6,2.6%
3,Harju maakond,0037,"MULTIPOLYGON (((504775.973 6570562.880, 504787...",Harju county,270490035,432784,13069164,20911,4.8,4.8%
4,Lääne maakond,0056,"MULTIPOLYGON (((460628.287 6512871.566, 460630...",Lääne county,113484461,181575,3417874,5469,3.0,3.0%


In [12]:
counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count.shp')

/var/folders/z4/9280v_g94039tvyzq2n4yrd40000gn/T/ipykernel_83139/556985071.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  counties.to_file('../../Thesis_2024_updates/Vector_Data/counties/counties_with_pixel_count_S1.shp')


In [13]:
counties_nice = counties.sort_values(by='percent_of_county', ascending=False)[['name', 'suitable_area_ha', 'county_area_ha', 'percent_of_county_str']]
counties_nice

,name,S1_suitable_area_ha,county_area_ha,percent_of_county_str
3,Harju county,20911,432784,4.8%
6,Ida-Viru county,10792,297177,3.6%
8,Tartu county,12043,334947,3.6%
13,Lääne-Viru county,13272,369538,3.6%
4,Lääne county,5469,181575,3.0%
5,Rapla county,8018,276423,2.9%
0,Saare county,8120,293832,2.8%
10,Võru county,7413,277317,2.7%
2,Hiiu county,2700,103258,2.6%
14,Põlva county,4788,182337,2.6%
